<a href="https://colab.research.google.com/github/sugarforever/LangChain-Advanced/blob/main/Integrations/AutoGen/autogen_langchain_uniswap_ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen + Flowise Use Case

**`AutoGen`** is a versatile framework that facilitates the creation of LLM applications by employing multiple agents capable of interacting with one another to tackle tasks. These AutoGen agents can be tailored to specific needs, engage in conversations, and seamlessly integrate human participation.

**`Flowise`** is an open source UI visual tool to build customized LLM flows using LangchainJS, written in Node Typescript/Javascript.

Flowise has been the leading no-code platform for Langchain based LLM workflow development. You may have already built great features on it.

In this tutorial, I will show you how to integrate Flowise flows into AutoGen. You will be able to empower your AutoGen agents with the capability of your Flowise flow, with minimal effort.

## Use Case - Uniswap Protocol AI Agent with Flowise Flow

`Uniswap` is a decentralized exchange that allows users to trade Ethereum-based tokens.

In this tutorial, I will walk you through the steps to simplify the AI agent we built in the previous tutorial - [AutoGen + LangChain Use Case - Uniswap Protocol AI Agent](./autogen_langchain_uniswap_ai_agent.ipynb). This agent can execute tasks that involve Uniswap knowledge.

We will need much less steps if we have already implemented the QA chain as a Flowise flow.

1. Fetch the Python code snippet for Flowise integration.
2. Define a function `answer_flowise_uniswap_question`.
3. Validate the `answer_flowise_uniswap_question` function.
4. Set up AutoGen user agent and assistant agent with function calling enabled.

  In the function mapping, the function defined in step 3 is changed to `answer_flowise_uniswap_question`.

### Environment Preparation

In [1]:
%pip install pyautogen~=0.1.0 docker -q -U

Note: you may need to restart the kernel to use updated packages.


In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)
#
# Sample content of OAI_CONFIG_LIST file below:
#
# [
#   {
#     "model": "gpt-4",
#     "api_key": "your openai api key"
#   }
# ]
#

### Steps

#### 1. Fetch the Python code snippet for Flowise integration

Click the API Endpoint on the top right corner of your Flowise canvas, and find the Python code snippet similar as below.

```python
import requests

API_URL = "http://localhost:4000/api/v1/prediction/433ed37e-9546-4e73-a688-7352b78bf852"

def query(payload):
    response = requests.post(API_URL, json=payload)
    return response.json()
    
output = query({
    "question": "Hey, how are you?",
})
```

We will reuse it in the AutoGen code base.

#### 2. Define a function `answer_flowise_uniswap_question`

It takes a parameter `question`, makes API call to the established Flowise flow, and answer it with the API response.

In [3]:
import requests

API_URL = "http://localhost:4000/api/v1/prediction/433ed37e-9546-4e73-a688-7352b78bf852"

def answer_flowise_uniswap_question(question):
  response = requests.post(API_URL, json={ "question": question })
  return response.json()

#### 3. Validate the `answer_flowise_uniswap_question` function

In [4]:
answer_flowise_uniswap_question("What are the main changes in Uniswap v3?")

'The main changes in Uniswap v3 include the removal of the need for users to track previous values of the accumulator externally, the introduction of multiple pools for each pair of tokens with different swap fees, the implementation of concentrated liquidity within a price range, the use of geometric mean TWAP instead of arithmetic mean TWAP, and the addition of a liquidity accumulator for external contracts to implement liquidity mining.'

#### 4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [5]:
llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_flowise_uniswap_question",
            "description": "Answer any Uniswap related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Uniswap protocol",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [6]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_flowise_uniswap_question": answer_flowise_uniswap_question}
)

### It's time to start a task for the agents.

Now, let's user the user agent to ask the agents to write an introduction blog for `Uniswap` protocol v3.

In [7]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.
"""
)

user_proxy (to assistant):


I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_flowise_uniswap_question *****
Arguments: 

{
  "question": "What is Uniswap?"
}
********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_flowise_uniswap_question...
user_proxy (to assistant):

***** Response from calling function "answer_flowise_uniswap_question" *****
Uniswap is a decentralized exchange protocol that allows users to trade ERC-20 tokens directly from their wallets. In Uniswap version 3, the main change is